Задание №3 по курсу ИТМО "Python для анализа данных"
Елькина Константина Анатольевича

In [246]:
import pandas as pd
import numpy as np
import pandas as pd

# Считываем датафрейм
df = pd.read_csv('train.csv')



In [247]:
# Выводим основную информацию о датасете: 
# информацию о типах данных, число пропусков, средние значения и т.д
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.48659

In [248]:
# Считаем процент выживаемости у каждого класса пассажиров
survival_rate = df.groupby('Pclass')['Survived'].mean() * 100
print(survival_rate)

Pclass
1    62.962963
2    47.282609
3    24.236253
Name: Survived, dtype: float64


In [249]:
def extract_maleName(name):
    """
    Функция для извлечения мужского имени из строки.
    Метод GitHub Copilot
    """
    firstName = name.split(',')[1].split()[1]
    
    return firstName

def extract_femaleName(name):
    """
    Функция для извлечения женского имени из строки.
    https://ru.stackoverflow.com/questions/1082775
    """
    
    firstName = name.str.extract(r"Mrs\.\s+[^(]*\((\w+)", expand=False)
    firstName.loc[firstName.isna()] = name.str.extract(r"\.\s+(\w+)", expand=False)
    
    return firstName

# Создаем отдельные DataFrame для мужчин и женщин
df_men = df[df['Sex'] == 'male'].copy()
df_women = df[df['Sex'] == 'female'].copy()

# Применяем соответствующие функции к столбцу 'Name' в каждом DataFrame
df_men['FirstName'] = df['Name'].apply(extract_maleName)
df_women['FirstName'] = extract_femaleName(df_women['Name'])

    
# print(df_men[['FirstName', 'Name']])
# print(df_women[['FirstName', 'Name']])

In [250]:
# Ищем самое популярное мужское и женское имя

popular_maleName = df_men['FirstName'].value_counts().idxmax()
popular_femaleName = df_women['FirstName'].value_counts().idxmax()

print("Самое популярное имя:", "мужское -", popular_maleName, "и женское -", popular_femaleName)

Самое популярное имя: мужское - William и женское - Anna


In [251]:
# Объединяем DataFrame c FirstName для мужчин и женщин
merged_df = pd.concat([df_men, df_women])

# Выведите самое популярное мужское и самое популярное женское имя на корабле в каждом классе
popular_names_by_class = merged_df.groupby(['Pclass', 'Sex'])['FirstName'].agg(lambda x: x.value_counts().idxmax())
print(popular_names_by_class)

Pclass  Sex   
1       female    Elizabeth
        male        William
2       female    Elizabeth
        male        William
3       female         Anna
        male        William
Name: FirstName, dtype: object


In [252]:
# Выводим часть датафрейма df с полями PassengerID, Name и Age пассажиов, возраст которых больше 44 лет
print(df[df['Age'] > 44][['PassengerId', 'Name', 'Age']])


     PassengerId                                               Name   Age
6              7                            McCarthy, Mr. Timothy J  54.0
11            12                           Bonnell, Miss. Elizabeth  58.0
15            16                   Hewlett, Mrs. (Mary D Kingcome)   55.0
33            34                              Wheadon, Mr. Edward H  66.0
52            53           Harper, Mrs. Henry Sleeper (Myna Haxtun)  49.0
..           ...                                                ...   ...
857          858                             Daly, Mr. Peter Denis   51.0
862          863  Swift, Mrs. Frederick Joel (Margaret Welles Ba...  48.0
871          872   Beckwith, Mrs. Richard Leonard (Sallie Monypeny)  47.0
873          874                        Vander Cruyssen, Mr. Victor  47.0
879          880      Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)  56.0

[115 rows x 3 columns]


In [253]:
# Выводим часть таблицы с пассажирами, возраст которых меньше 44 лет и которые мужского пола
print(df[(df['Age'] < 44) & (df['Sex'] == 'male')][['PassengerId', 'Name', 'Sex', 'Age']])

     PassengerId                            Name   Sex   Age
0              1         Braund, Mr. Owen Harris  male  22.0
4              5        Allen, Mr. William Henry  male  35.0
7              8  Palsson, Master. Gosta Leonard  male   2.0
12            13  Saundercock, Mr. William Henry  male  20.0
13            14     Andersson, Mr. Anders Johan  male  39.0
..           ...                             ...   ...   ...
883          884   Banfield, Mr. Frederick James  male  28.0
884          885          Sutehall, Mr. Henry Jr  male  25.0
886          887           Montvila, Rev. Juozas  male  27.0
889          890           Behr, Mr. Karl Howell  male  26.0
890          891             Dooley, Mr. Patrick  male  32.0

[368 rows x 4 columns]


In [254]:
# Приводим в порядок значения в столбце 'Cabin'
df['Cabin'] = pd.to_numeric(df['Cabin'], errors='coerce').fillna(0).astype(int)

# Определяем многоместные каюты
multi_room = df[df['Cabin'] > 1]

# Выводим количество многоместных кают
print(multi_room.value_counts().sum)

<bound method Series.sum of Series([], Name: count, dtype: int64)>
